Import the USSCS database kernel

In [1]:
from db_kernel import manage
from db_kernel import user as user_kernel
import qrcode
import rsa
#without login:
def add_user(username:str, password:str, public_key:rsa.PublicKey, private_key:rsa.PrivateKey, privacy:int=0, twofa:bool=False):
    """
    Add a user to the database.
    If two factor authentication is enabled, will return qrcode image.
    """
    return manage().add_user(username, password, public_key, private_key, privacy, twofa)

#with login:
class user:
    def __init__(self, username:str, passwort:str, twofa_key:str=None):
        self._user = user_kernel(username, passwort, twofa_key)
        self._manage = manage()
    
    def send_message(self, target:str, message:str, type:str='text'):
        """
        Send a message to a user.
        """
        return self._user.send_message(target, message, type)
    def get_messages(self, target:str, message_id:int=-1):
        """
        Get all messages from a user.
        id is the id of the last message you have.
        """
        return self._user.get_conversation(target, message_id)
    def get_users_with_unread_messages(self):
        """
        Get all users with unread messages.
        """
        return self._user.get_unread_messages()
    
    def add_contact(self, contact:str):
        """
        Add a contact.
        """
        return self._user.add_contact(contact)
    def remove_contact(self, contact:str):
        """
        Remove a contact.
        """
        return self._user.remove_contact(contact)
    def get_contacts(self):
        """
        Get all contacts.
        """
        return self._manage.get_contacts(self._user.username)
    
    def set_privacy(self, privacy:int):
        """
        Set privacy level.
        0: public
        1: contacts
        """
        return self._user.set_privacy(privacy)
    def get_privacy(self):
        """
        Get privacy level.
        0: public
        1: contacts
        """
        return self._user.privacy
    
    def enable_twofa(self) -> qrcode.image:
        """
        Enable two factor authentication.
        """
        return self._user.enable_2fa()
    def disable_twofa(self):
        """
        Disable two factor authentication.
        """
        return self._user.disable_2fa()
    def get_twofa(self):
        """
        Get two factor authentication status.
        """
        return self._user.is_2fa_enabled

    def delete_account(self):
        """
        Delete account.
        """
        return self._user.delete_account()
    
    def get_all_users(self):
        """
        Get all users.
        """
        return self._manage.get_all_users()
    
    def change_password(self, new_password:str):
        """
        Change password.
        """
        return self._user.change_password(new_password)
    


In [2]:
#delete all .db files
import glob 
import os
for file in glob.glob("*.db"):
    os.remove(file)
pub, priv = rsa.newkeys(512)
pub2, priv2 = rsa.newkeys(512)
img = add_user('test', 'test', pub, priv, 0, False)
add_user('test2', 'test2', pub2, priv2, 0, False)
user1 = user('test', 'test')
user2 = user('test2', 'test2')


Send messages

In [3]:

user1.send_message('test2', 'test')
user2.send_message('test', 'test2')

Read messages

In [4]:
print(user1.get_users_with_unread_messages())
print(user2.get_users_with_unread_messages())
      
print(user1.get_messages('test2'))
print(user2.get_messages('test'))


['test2']
['test']
[('test', b'test', '20:12 09/04/23', 0, 'text'), ('test2', b'test2', '20:12 09/04/23', 0, 'text')]
[('test', b'test', '20:12 09/04/23', 0, 'text'), ('test2', b'test2', '20:12 09/04/23', 1, 'text')]


Privacy testing

In [5]:
user1.set_privacy(1)
#user2.send_message('test', 'test3') #should not work (get error)
user1.add_contact('test2')
user2.send_message('test', 'test3') #should work


2FA testing

In [6]:
img = user1.enable_twofa()
img.save('qrcode.png')
user1 = user("test", "test", input("2fa key: "))


ValueError: 2FA is enabled